## Create table to postgresql database: olist_db

First Method using sqlalchemy unable to set primary

In [34]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, VARCHAR, TIMESTAMP, TEXT, INTEGER
from sqlalchemy import text

username='postgres'
password = 'admin'
hostname = 'localhost'
port = 5432
database ='olist_db'

db_url = f'postgresql://{username}:{password}@{hostname}:{port}/{database}'

# Create engine
engine = create_engine(db_url)


# File path and table name
csv_file = "clean_data/cleaned_orders_reviews.csv"
table_name = "orders_reviews_tbl"


# Read CSV into DataFrame
clean_orders_reviews_df = pd.read_csv(csv_file)


# Define data types for each column
dtype = {
    'review_id': VARCHAR,  # Example: VARCHAR with length 50
    'order_id': VARCHAR,
    'review_score': INTEGER,
    'review_comment_title': TEXT,
    'review_comment_message': TEXT,
    'review_creation_date': TIMESTAMP,
    'review_answer_timestamp' : TIMESTAMP,
    'cleaned_review_comment': TEXT,
    'translated_review_comment': TEXT,
    'translated_comment_title': TEXT

}


# Upload DataFrame to PostgreSQL
clean_orders_reviews_df.to_sql(table_name, engine, if_exists='replace', dtype=dtype, index=False)

# Add the constraints to the table
with engine.connect() as connection:

    add_pk_sql = text(
        "ALTER TABLE orders_reviews_tbl "
        "ADD CONSTRAINT pk_review_id PRIMARY KEY (review_id, order_id);"
    )  
    # Execute the SQL statement
    connection.execute(add_pk_sql)

    # alter_order_id_fk = text("ALTER TABLE orders_reviews_tbl ADD CONSTRAINT fk_order_id FOREIGN KEY (order_id) REFERENCES order_tbl(order_id);")

    # # Execute the SQL statement
    # connection.execute(alter_order_id_fk)

    # Commit the transaction
    connection.commit()

C:\Users\T470\AppData\Local\Temp\ipykernel_11324\2515822291.py:24: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  clean_orders_reviews_df = pd.read_csv(csv_file)


ProgrammingError: (psycopg2.errors.UndefinedTable) relation "order_tbl" does not exist

[SQL: ALTER TABLE orders_reviews_tbl ADD CONSTRAINT fk_order_id FOREIGN KEY (order_id) REFERENCES order_tbl(order_id);]
(Background on this error at: https://sqlalche.me/e/20/f405)

## 2nd method using psycopg2

In [25]:
import psycopg2
from psycopg2 import sql
import pandas as pd

# File path and table name
csv_file = "clean_data/cleaned_orders_reviews.csv"
table_name = "orders_reviews_tbl"


# Read CSV into DataFrame
clean_orders_reviews_df = pd.read_csv(csv_file)

# # Check for duplicate review_id values
# duplicates = clean_orders_reviews_df['review_id'].duplicated().any()

# if duplicates:
#     print("Error: DataFrame contains duplicate review_id values.")
#     # Handle the error or remove duplicates from the DataFrame
# else:
#     print("No duplicate review_id values found. Proceeding with insertion.")
#     # Proceed with the insertion process

conn = psycopg2.connect(
    dbname="olist_db",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)

# Create a cursor object
cur = conn.cursor()


# Create a table query
create_table_query = """
    CREATE TABLE IF NOT EXISTS orders_reviews_tbl (
        review_id VARCHAR,
        order_id VARCHAR,
        review_score INT,
        review_comment_title TEXT,
        review_comment_message TEXT,
        review_creation_date TIMESTAMP,
        review_answer_timestamp TIMESTAMP,
        cleaned_review_comment TEXT,
        translated_review_comment TEXT,
        translated_comment_title TEXT,
        PRIMARY KEY (review_id, order_id)
        -- FOREIGN KEY (order_id) REFERENCES orders_tbl(order_id)
    );
"""


# Execute the table creation query
cur.execute(create_table_query)

# Commit the transaction
conn.commit()

# Assuming orders_reviews_df is your DataFrame
# Convert DataFrame to list of tuples, including the timestamp
data = [(row['review_id'], row['order_id'], row['review_score'], row['review_comment_title'], row['review_comment_message'], row['review_creation_date'], row['review_answer_timestamp'], row['cleaned_review_comment'],row['translated_review_comment'], row['translated_comment_title']) for index, row in clean_orders_reviews_df.iterrows()]


# Create an insert query
insert_query = sql.SQL("""
    INSERT INTO orders_reviews_tbl (review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp, cleaned_review_comment, translated_review_comment, translated_comment_title)
    VALUES {};
""").format(
    sql.SQL(',').join(map(sql.Literal, data))
)

# Execute the insert query
cur.execute(insert_query)

# Commit the transaction
conn.commit()

# # Close the cursor and connection
# cur.close()
# conn.close()

C:\Users\T470\AppData\Local\Temp\ipykernel_11324\3301257580.py:11: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  clean_orders_reviews_df = pd.read_csv(csv_file)


In [24]:
clean_orders_reviews_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,cleaned_review_comment,translated_review_comment,translated_comment_title
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,NaN,NaN,NaN
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,NaN,NaN,NaN
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,NaN,NaN,NaN
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,Recebi bem antes do prazo estipulado.,I received it well before the stipulated deadl...,NaN
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,Parabéns lojas lannister adorei comprar pela I...,Congratulations lannister stores I loved shopp...,NaN


In [ ]:
# Close the cursor and connection
# cursor.close()
# conn.close()